In [1]:
import pulp # pulp

In [2]:
s = [4, 5, 3, 5, 7, 1, 0, 3, 2, 10]  # durations of the jobs 
r = [3, 4, 7, 11, 10, 0, 0, 10, 0, 15] # release time
d = [11, 12, 20, 25, 20, 10, 30, 30, 10, 20] # duration
c = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] # cost per job per hour over due
M = 100000 # large M
n = len(s)  # number of jobs

In [3]:
sms = pulp.LpProblem(name="sms", sense=pulp.LpMinimize) # define problem

In [4]:
# decision variables
x = [pulp.LpVariable(name=f'x_{i}', lowBound=0, cat='Continuous') for i in range(n)]
y = [[pulp.LpVariable(name=f'y_{i},{j}', cat='Binary') for j in range(n)] for i in range(n)]
z = [pulp.LpVariable(name=f'z_{i}', lowBound=0, cat='Continuous') for i in range(n)] # job tradiness

In [5]:
sms += pulp.lpSum(z), 'sum_of_finish_times' # objective function

In [6]:
# subject to constraints:

for i in range(n):
    sms += x[i] >= 0, f'start_time_{i}_>_0'
    sms += x[i] >= r[i] , f'r: job_{i}_should_start_after_{r[i]}'
    sms += z[i] >= x[i] + s[i] - d[i] , f'tardiness_{i}'
    sms += z[i] >= 0, f'tardiness_{i}_>_0'
    
for i in range(n):
    for j in range(n):
        if i is not j:
            sms += (y[i][j] + y[j][i]) == 1, f'single-machine_not_{i}_and_{j}_simultaneous'
            sms += x[i] + s[i] <= x[j] + M * (1 - y[i][j]), f'start_{j}_after_{i}'

In [7]:
sms.solve() # solve model
print("Optimization status:", pulp.LpStatus[sms.status]) # print status
print('The optimal starting times are ', [x[i].value() for i in range(n)]) # starting times as solved by model
print('with objective value of', sms.objective.value()) # total tardiness

Optimization status: Optimal
The optimal starting times are  [3.0, 7.0, 19.0, 22.0, 12.0, 2.0, 2.0, 27.0, 0.0, 30.0]
with objective value of 24.0


In [8]:
sms

sms:
MINIMIZE
1*z_0 + 1*z_1 + 1*z_2 + 1*z_3 + 1*z_4 + 1*z_5 + 1*z_6 + 1*z_7 + 1*z_8 + 1*z_9 + 0
SUBJECT TO
start_time_0_>_0: x_0 >= 0

r:_job_0_should_start_after_3: x_0 >= 3

tardiness_0: - x_0 + z_0 >= -7

tardiness__0_>_0: z_0 >= 0

start_time_1_>_0: x_1 >= 0

r:_job_1_should_start_after_4: x_1 >= 4

tardiness_1: - x_1 + z_1 >= -7

tardiness__1_>_0: z_1 >= 0

start_time_2_>_0: x_2 >= 0

r:_job_2_should_start_after_7: x_2 >= 7

tardiness_2: - x_2 + z_2 >= -17

tardiness__2_>_0: z_2 >= 0

start_time_3_>_0: x_3 >= 0

r:_job_3_should_start_after_11: x_3 >= 11

tardiness_3: - x_3 + z_3 >= -20

tardiness__3_>_0: z_3 >= 0

start_time_4_>_0: x_4 >= 0

r:_job_4_should_start_after_10: x_4 >= 10

tardiness_4: - x_4 + z_4 >= -13

tardiness__4_>_0: z_4 >= 0

start_time_5_>_0: x_5 >= 0

r:_job_5_should_start_after_0: x_5 >= 0

tardiness_5: - x_5 + z_5 >= -9

tardiness__5_>_0: z_5 >= 0

start_time_6_>_0: x_6 >= 0

r:_job_6_should_start_after_0: x_6 >= 0

tardiness_6: - x_6 + z_6 >= -30

tardiness_